In [1]:
from agents import Agent, function_tool, Runner

In [2]:
import requests

In [3]:
runner = Runner()


In [4]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()


In [5]:
runner = OpenAIAgentsSDKRunner(
    chat_interface = chat_interface,
    agent = assistant
)

NameError: name 'assistant' is not defined

### Youtube

In [6]:
import youtube

In [18]:
subtitles = youtube.fetch_transcript_cached('vK_SxyqIfwk')

In [14]:
def fetch_youtube_transcript(video_id: str) -> str:
    """
    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.

    Args:
        video_id (str): The unique YouTube video ID.

    Returns:
        str: The subtitles generated from the video's transcript.
    """
    return youtube.fetch_transcript_cached(video_id)


In [15]:
summary_instructions = """
You're a helpful assistant that summarizes YouTube videos
"""

tools = [
    function_tool(fetch_youtube_transcript)
]

summary_agent = Agent(
    name='summary agent',
    tools=tools,
    handoff_description="Whenever the user needs a summary of the video",
    instructions=summary_instructions,
    model='gpt-4o-mini'
)

In [16]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=summary_agent
)

await runner.run();

You: stop


Chat ended.


### search agent

In [13]:
from minsearch import AppendableIndex

In [17]:
import docs

In [24]:
index = AppendableIndex(text_fields=['content'])

In [61]:
from minsearch import AppendableIndex
import docs
from typing import Any, Dict, List, Optional

class SearchTools:

    def __init__(self) -> None:
        self.index = AppendableIndex(text_fields=['content'])

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the index for documents matching a query string.

        Args:
            query (str): The search query.

        Returns:
            List[Dict[str, Any]]: A list of search result dictionaries.
        """
        return self.index.search(query, num_results=5)

    def index_content(self, video_id: str, content: Optional[str] = None) -> str:
        """
        Index text content from a given source into the search index.

        Args:
            video_id (str): video_id or link to the web page.
            content (str): transcript for the provided video_id or None if the transcript needs to be downloaded

        Returns:
            str: "SUCCESS" upon successful indexing.
        """
        if content is None:
            content = fetch_youtube_transcript(video_id)
        chunks = docs.sliding_window(content, size=3000, step=1500)

        for chunk in chunks:
            chunk["video_id"] = video_id
            self.index.append(chunk)

        return "SUCCESS"

In [62]:
search_tools= SearchTools()

In [63]:
search_tools.index_content('vK_SxyqIfwk')

'SUCCESS'

In [64]:
search_tools.search('Tesla')

[{'start': 0,
  'content': "0:00 Hey everyone, welcome to our event. This\n0:02 event is brought to you by data talks\n0:03 club which is a community of people who\n0:05 love data. We have weekly events today.\n0:08 Uh this is one of such events. Um if you\n0:11 want to find out more about the events\n0:13 we have, there is a link in the\n0:14 description. Um so click on that link,\n0:16 check it out right now. We actually have\n0:19 quite a few events in our pipeline, but\n0:21 we need to put them on the website. Uh\n0:24 but keep an eye on it anyways. Um so we\n0:27 will put um them. you'll see them. And\n0:30 then don't forget to subscribe to our\n0:32 YouTube channel. So this is the most um\n0:35 reliable way of getting notified when\n0:37 our stream starts. And last but not\n0:40 least, do not forget to join our um data\n0:43 community where you can hang out with\n0:45 data with other data enthusiasts. During\n0:48 today's interview, you can ask any\n0:49 question you want. There 

In [65]:
search_tools = SearchTools()

search_tool_methods = [
    function_tool(search_tools.search),
    function_tool(search_tools.index_content)
]

In [66]:
search_instructions = """
Your task is to search through indexed documents.

Before performing a search:
1. Check if the document has been indexed.
2. If not, call `index_content` to index it.

Important rules:
- Do NOT call `search` using a YouTube ID, webpage URL, or any direct link as the query.
- After successfully indexing a source, do NOT perform a search automatically.
- Instead, tell the user something like:
  "The video (or page) has been indexed. What would you like to learn about it?"

Only perform a search if the user provides a natural-language question or topic of interest
(e.g., "What does the video say about AI safety?"), not a URL or ID.
Do not attempt to summarize content.
"""

search_agent = Agent(
    name='search_agent',
    handoff_description='use this agent when you need to perform search in the knowledge base',
    tools=search_tool_methods,
    instructions=search_instructions,
    model='gpt-4o-mini'
)